In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

harshvardhinip_more_nlp_path = kagglehub.dataset_download('harshvardhinip/more-nlp')

print('Data source import complete.')


In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Tokenize text
text = "Natural language processing is fun!"
tokens = nltk.word_tokenize(text)
print(tokens)

# Compute BLEU score for evaluation
reference = [["sarcasm", "is", "not", "obvious"]]
candidate = ["sarcasm", "is", "subtle"]
bleu_score = sentence_bleu(reference, candidate)
print(f"BLEU Score: {bleu_score}")


['Natural', 'language', 'processing', 'is', 'fun', '!']
BLEU Score: 0.5444460596606694


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
import torch
import pandas as pd
import numpy as np
import pickle
from transformers import BartTokenizer, BartForConditionalGeneration, ViTModel
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BartTokenizer, BartForConditionalGeneration, ViTModel
import pandas as pd
import pickle
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms


### Preprocessing

In [ ]:

DATA_PATH = "/kaggle/input/more-nlp/MORE-PLUS-DATASET/"

df_train = pd.read_csv(DATA_PATH + "train_df.tsv", sep='\t')
df_valid = pd.read_csv(DATA_PATH + "val_df.tsv", sep='\t')

with open(DATA_PATH + "D_train.pkl", 'rb') as f:
    image_descriptions_train = pickle.load(f)

with open(DATA_PATH + "O_train.pkl", 'rb') as f:
    detected_objects_train = pickle.load(f)


In [ ]:
df_train

,Unnamed: 0,pid,text,explanation,target_of_sarcasm
0,0,931874353976938497,'check out the pulse pounding excitement of # ...,the people are bored of waiting in line <num> ...,waiting in line
1,1,880425829246922752,'proud to have a <user> who attacks private ci...,it's sad to have a <user> who attacks private ...,<user> for attacking citizens via twitter
2,2,690915881082343424,'the view from my house ........ yay # pasnow...,the author has a terrible view of everything c...,blizzard
3,3,915228456757059585,'these # colombo # jaffna # vavuniya luxury bu...,these # colombo # jaffna # vavuniya luxury bus...,technology in vavuniya luxury busses
4,4,494194068998468686_25639236,"Thanks North Carolina, I was worried we weren'...",the author was worried that they were gonna hi...,traffic in north carolina
...,...,...,...,...,...
2978,2978,806820636517691392,'well done # us # syria',US shouldn't have attacked hospital in mosul a...,US's attack on hospital in mosul
2979,2979,824024594755518464,oh yes fios let 's upgrade our wi-fi for even...,the author isn't interested to upgrade their w...,slower wifi
2980,2980,720315804189732864,'inspiring confidence during the registration ...,The registration website doesn’t seem very sec...,<user>'s registraion process ux
2981,2981,992845208666844469_180900586,Don't I feel #safe in our #government offices....,it's unsafe to have such wires laying around.,safety in government offices


In [ ]:
def preprocess_text(df):
    texts = df['text'].tolist()
    explanations = df['explanation'].tolist()
    sarcasm_targets = df['target_of_sarcasm'].tolist()
    return texts, explanations, sarcasm_targets

train_texts, train_explanations, train_targets = preprocess_text(df_train)
valid_texts, valid_explanations, valid_targets = preprocess_text(df_valid)


In [ ]:
train_texts

["'check out the pulse pounding excitement of # paxunplugged of waiting in line <num> hours before game sign up begins . waiting in line , what a gaming convention is all about . '",
 "'proud to have a <user> who attacks private citizens via twitter instead of doing hard work on health care , nat sec the job requires '",
 "'the view from my house ........ yay  # pasnow # blizzard2016'",
 "'these # colombo # jaffna # vavuniya luxury busses are very high tech  # cultureshock # turkishairlines'",
 "Thanks North Carolina, I was worried we weren't gonna hit traffic",
 "'junk mail : free viagra . stripper wants to meet me . and , someone will send me 100k if i give them my account number . score ! # japan '",
 "'god awful venue . terrible view . '",
 "'kids : - take selfie from mobiles adults : - take selfie from dslr legends : - take selfie from landline # amitabhbachchan # trending '",
 "i hope england isn 't too rainy , or anything ...  emoji_131 emoji_15",
 '\'your persistence has paid o

In [ ]:
train_explanations

['the people are bored of waiting in line <num> hours before game sign up begins.',
 "it's sad to have a <user> who attacks private citizens via twitter instead of doing hard work on health care, nat sec the job requires.",
 'the author has a terrible view of everything covered with snow from his house.',
 "these # colombo # jaffna # vavuniya luxury busses aren't very high tech.",
 'the author was worried that they were gonna hit traffic in North Carolina and it turned out to be true.',
 'the author is annoyed with such junk mail.',
 "it's a beautiful venue and lovely view.",
 "one can't take a selfie from a landline.",
 'england is too rainy.',
 "their persistence hasn't paid off, the author doesn't believe that the earth is flat.",
 "these toddlers aren't helpful at building the puzzle.",
 'moving is exhausting and not fun.',
 "the author expected this since she wasn't sure that clinton was always honest and trustworthy.",
 "it's a wrong priority to status update than to call police 

### Architecture models

In [ ]:
#Extracting high-level image features using a Vision Transformer
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224')

def extract_image_features(image_path):
    image = preprocess_image(image_path)  # Define a function for image preprocessing
    with torch.no_grad():
        features = vit_model(image.unsqueeze(0)).last_hidden_state
    return featuresA


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
df_train = pd.read_csv('/kaggle/input/more-nlp/MORE-PLUS-DATASET/train_df.tsv', sep='\t')
df_valid = pd.read_csv('/kaggle/input/more-nlp/MORE-PLUS-DATASET/val_df.tsv', sep='\t')

with open('/kaggle/input/more-nlp/MORE-PLUS-DATASET/D_train.pkl', 'rb') as f:
    image_descriptions_train = pickle.load(f)
with open('/kaggle/input/more-nlp/MORE-PLUS-DATASET/O_train.pkl', 'rb') as f:
    detected_objects_train = pickle.load(f)

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
vision_model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
class SharedFusion(nn.Module):
    def __init__(self, text_dim=768, img_dim=768, fusion_dim=768):
        super(SharedFusion, self).__init__()
        self.text_proj = nn.Linear(text_dim, fusion_dim)
        self.img_proj = nn.Linear(img_dim, fusion_dim)
        self.fusion_layer = nn.Linear(fusion_dim * 2, fusion_dim)
        self.activation = nn.ReLU()

    def forward(self, text_features, image_features):
        text_emb = self.activation(self.text_proj(text_features))
        img_emb = self.activation(self.img_proj(image_features))
        fused_emb = torch.cat((text_emb, img_emb), dim=-1)
        return self.activation(self.fusion_layer(fused_emb))


shared_fusion = SharedFusion().to(device)  # Move fusion module to GPU

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])

In [ ]:
class TurboModel(nn.Module):
    def __init__(self, bart_model, fusion_module):
        super(TurboModel, self).__init__()
        self.bart = bart_model
        self.fusion = fusion_module

    def forward(self, input_ids, attention_mask, labels, image_features):
        with torch.no_grad():
            text_embeddings = self.bart.model.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state.mean(dim=1)
        fused_features = self.fusion(text_embeddings, image_features.squeeze(1))

        encoder_outputs = self.bart.model.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        outputs = self.bart(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            encoder_outputs=encoder_outputs
        )
        return outputs

### MUsE preprocessing

In [ ]:
from torchvision import transforms
from PIL import Image
import os

class MuSEDataset(Dataset):
    def __init__(self, dataframe, image_folder, image_desc, detected_objs, tokenizer, vit_model, max_len=128):
        self.dataframe = dataframe
        self.image_folder = image_folder  # path to actual images
        self.image_desc = image_desc      # text-based image descriptions
        self.detected_objs = detected_objs
        self.tokenizer = tokenizer
        self.vit_model = vit_model
        self.max_len = max_len

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        explanation = self.dataframe.iloc[idx]['explanation']
        sarcasm_target = self.dataframe.iloc[idx]['target_of_sarcasm']

        pid = self.dataframe.iloc[idx]['pid']

        # Load actual image from folder
        image_path = os.path.join(self.image_folder, f"{pid}.jpg")  # or .png depending on extension
        try:
            image = Image.open(image_path).convert('RGB')
            image_tensor = self.transform(image).unsqueeze(0).to(device)
            with torch.no_grad():
                image_features = self.vit_model(image_tensor).last_hidden_state.mean(dim=1).squeeze(0)
        except Exception as e:
            image_features = torch.zeros(768, device=device)  # fallback if image missing

        # Object-level features
        detected_objects = self.detected_objs.get(pid, torch.zeros(768, device=device))

        # Image text description (for concatenation)
        image_description_text = self.image_desc.get(pid, "")

        # Build enriched text input: text + description + target
        enriched_text = f"{text} [DESC]: {image_description_text} [TARGET]: {sarcasm_target}"

        # Tokenize enriched input
        text_enc = self.tokenizer(enriched_text, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')
        exp_enc = self.tokenizer(explanation, truncation=True, padding='max_length', max_length=self.max_len, return_tensors='pt')

        input_ids = text_enc['input_ids'].squeeze(0).to(device)
        attention_mask = text_enc['attention_mask'].squeeze(0).to(device)
        labels = exp_enc['input_ids'].squeeze(0).to(device)

        with torch.no_grad():
            text_embeddings = bart_model.get_encoder()(input_ids.unsqueeze(0)).last_hidden_state.mean(dim=1)

        # Shared fusion of text and image features
        fused_features = shared_fusion(text_embeddings, image_features.unsqueeze(0))

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels,
            'image_features': fused_features
        }


In [ ]:
turbo_model = TurboModel(bart_model, shared_fusion).to(device)

In [ ]:
train_dataset = MuSEDataset(df_train,'/kaggle/input/more-nlp/MORE-PLUS-DATASET/images' ,image_descriptions_train, detected_objects_train, tokenizer, vision_model)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training Loop
optimizer = optim.AdamW(turbo_model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
print(train_dataset.dataframe.columns)




Index(['Unnamed: 0', 'pid', 'text', 'explanation', 'target_of_sarcasm'], dtype='object')


In [ ]:
train_dataset[2]

{'input_ids': tensor([    0,    75,   700,  1217,    31,   127,   790, 47965,  1423,   857,
          1437,   849,  6977,  8310,   849,  3089, 29549,  9029,   108,   646,
         40429,   347, 42645,    89,    32,    80, 23439,    29,    14,    32,
          2934,    11,     5,  1958,   646,   565, 40325, 42645,  3089, 29549,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

### Training

In [ ]:
valid_dataset = MuSEDataset(df_valid, '/kaggle/input/more-nlp/MORE-PLUS-DATASET/images', image_descriptions_train, detected_objects_train, tokenizer, vision_model)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [ ]:

def train(model, dataloader, optimizer, criterion, epochs=3):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()

            # Move batch data to GPU
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            image_features = batch['image_features'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                image_features=image_features
            )
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

        # Save Model Checkpoint
        checkpoint_path = f"bart_model_epoch_{epoch+1}.pt"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Model checkpoint saved at {checkpoint_path}")

# Start Training
train(turbo_model, train_loader, optimizer, criterion, epochs=5)


Epoch 1, Loss: 1.7423279088608083
Model checkpoint saved at bart_model_epoch_1.pt
Epoch 2, Loss: 0.2291044712385392
Model checkpoint saved at bart_model_epoch_2.pt
Epoch 3, Loss: 0.19481460582763754
Model checkpoint saved at bart_model_epoch_3.pt
Epoch 4, Loss: 0.1802091072587406
Model checkpoint saved at bart_model_epoch_4.pt
Epoch 5, Loss: 0.15052080341679527
Model checkpoint saved at bart_model_epoch_5.pt


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1b9359dda33b6da398c278a935b5e85ef5397c73c7b020ea81109cb5519c6702
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip uninstall preprocessing -y
!pip install --upgrade nltk


Found existing installation: preprocessing 0.1.13
Uninstalling preprocessing-0.1.13:
  Successfully uninstalled preprocessing-0.1.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.9 MB/s eta 0:00:0000:010:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [ ]:
!pip install bert_score

In [ ]:
# !rm -rf /kaggle/working/nltk_data


In [ ]:
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.download('omw-1.4', download_dir='/kaggle/working/nltk_data')

# Then tell nltk to look there
import nltk.data
nltk.data.path.append('/kaggle/working/nltk_data')


[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data...


In [ ]:
# Unzip to the correct nltk_data structure
!unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora


Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
   creating: /kaggle/working/nltk_data/corpora/wordnet/
  inflating: /kaggle/working/nltk_data/corpora/wordnet/lexnames  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adv  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/README  
  inflating: /kaggle/w

In [ ]:
from nltk.corpus import wordnet

print(wordnet.synsets("sarcasm"))


[Synset('sarcasm.n.01')]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('punkt_tab', download_dir="/kaggle/working/nltk_data")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
turbo_model.load_state_dict(torch.load("bart_model_epoch_5.pt", map_location=device))
turbo_model.to(device)
turbo_model.eval()


<ipython-input-58-9470e90f1d5f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  turbo_model.load_state_dict(torch.load("bart_model_epoch_5.pt", map_location=device))


TurboModel(
  (bart): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features

In [ ]:

from evaluate import load

# Load evaluation metrics
rouge = load("rouge")
bleu = load("bleu")
meteor = load("meteor")
bertscore = load("bertscore")

# Function to generate predictions
def generate_predictions(model, dataloader, tokenizer):
    predictions, references = [], []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            inputs = batch["input_ids"].to(device)
            labels = batch["labels"].to(device)

            outputs =  model.bart.generate(inputs, max_length=50)
            pred_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            ref_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

            predictions.extend(pred_texts)
            references.extend(ref_texts)

    return predictions, references

# Function to compute evaluation metrics
def compute_metrics(predictions, references):
    # Compute individual metrics
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="en")

    # Compute BLEU scores manually for B1–B4
    bleu_scores = {}
    for i in range(1, 5):
        bleu_result = bleu.compute(predictions=predictions, references=references, max_order=i)
        bleu_scores[f"BLEU-{i}"] = bleu_result["bleu"]

    # BERTScore returns precision, recall, f1 for each pair – take average F1
    bert_f1_avg = sum(bert_scores["f1"]) / len(bert_scores["f1"])

    return {
        "ROUGE": rouge_scores,
        "METEOR": meteor_score["meteor"],
        **bleu_scores,
        "BERTScore_F1": bert_f1_avg
    }

# 🔹 Generate predictions
# preds, refs = generate_predictions(turbo_model, valid_loader, tokenizer)

# # 🔹 Compute metrics
# results = compute_metrics(preds, refs)
# print(results)


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def train(model, train_loader, valid_loader, tokenizer, optimizer, criterion, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            optimizer.zero_grad()

            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            image_features = batch['image_features']

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                image_features=image_features
            )

            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"\nEpoch {epoch + 1}/{epochs} - Training Loss: {avg_loss}")

        # 🧪 Evaluate on validation set
        model.eval()
        with torch.no_grad():
            preds, refs = generate_predictions(model, valid_loader, tokenizer)
            metrics = compute_metrics(preds, refs)

        print(f"\nEvaluation Metrics (Epoch {epoch + 1}):")
        print("-" * 50)

        # Print ROUGE
        print(f"ROUGE-1: {metrics['ROUGE']['rouge1']}")
        print(f"ROUGE-2: {metrics['ROUGE']['rouge2']}")
        print(f"ROUGE-L: {metrics['ROUGE']['rougeL']}")

        # Print BLEU
        for i in range(1, 5):
            print(f"BLEU-{i}: {metrics[f'BLEU-{i}']}")

        # Print METEOR and BERTScore
        print(f"METEOR: {metrics['METEOR']}")
        print(f"BERTScore-F1: {metrics['BERTScore_F1']}")
        print("-" * 50)

        # 💾 Save model checkpoint
        torch.save(model.state_dict(), f"turbo_model_epoch_{epoch + 1}.pt")
        print(f"Model checkpoint saved: turbo_model_epoch_{epoch + 1}.pt")


In [ ]:
train(
    model=turbo_model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    tokenizer=tokenizer,
    optimizer=optimizer,
    criterion=criterion,
    epochs=5
)



Epoch 1/5 - Training Loss: 0.20754970116092558

Evaluation Metrics (Epoch 1):
--------------------------------------------------
ROUGE-1: 0.548678741553891
ROUGE-2: 0.39015285583093173
ROUGE-L: 0.5216460339944049
BLEU-1: 0.5301434697120316
BLEU-2: 0.4492918439659864
BLEU-3: 0.39472951759850156
BLEU-4: 0.3529896819005299
METEOR: 0.5380490878147943
BERTScore-F1: 0.9245555908339365
--------------------------------------------------
Model checkpoint saved: turbo_model_epoch_1.pt

Epoch 2/5 - Training Loss: 0.1783724139160651

Evaluation Metrics (Epoch 2):
--------------------------------------------------
ROUGE-1: 0.5560470713237173
ROUGE-2: 0.3963856638939367
ROUGE-L: 0.5304922275789619
BLEU-1: 0.5550711572306943
BLEU-2: 0.47246316293823537
BLEU-3: 0.4155539969868244
BLEU-4: 0.3717455941877607
METEOR: 0.5555059321663439
BERTScore-F1: 0.9248437925747462
--------------------------------------------------
Model checkpoint saved: turbo_model_epoch_2.pt

Epoch 3/5 - Training Loss: 0.153323185

### Testing

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import pandas as pd
import pickle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_trained_model(model_path):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    fusion = SharedFusion().to(device)
    model = TurboModel(bart, fusion).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model, tokenizer

In [ ]:
def load_test_data(tsv_path, d_path, o_path):
    df = pd.read_csv(tsv_path, sep='\t')
    with open(d_path, 'rb') as f:
        d_features = pickle.load(f)
    with open(o_path, 'rb') as f:
        o_features = pickle.load(f)
    return df, d_features, o_features

In [ ]:


def generate_explanations(model, tokenizer, inputs, max_len=50):
    predictions = []
    batch_size = 8

    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i+batch_size]
        encoded = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.bart.generate(**encoded, max_length=max_len)  # use BART's generate method
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(decoded)

    return predictions

def test_inference(model_path, tsv_path, d_path, o_path, output_file="predictions.txt", paired_output_file="pred_vs_actual.txt"):
    print(f"Using device: {device}")

    model, tokenizer = load_trained_model(model_path)

    df, d_features, o_features = load_test_data(tsv_path, d_path, o_path)
    inputs = df["text"].tolist()
    references = df["explanation"].tolist() if "explanation" in df.columns else None

    predictions = generate_explanations(model, tokenizer, inputs)

    with open(output_file, "w") as f:
        for pred in predictions:
            f.write(pred.strip() + "\n")
    print(f"Predictions saved to {output_file}")

    if references:
        with open(paired_output_file, "w") as f:
            for actual, pred in zip(references, predictions):
                f.write(f"[ACTUAL]:  {actual.strip()}\n")
                f.write(f"[PREDICT]: {pred.strip()}\n")
                f.write("-" * 50 + "\n")
        print(f"Paired output saved to {paired_output_file}")

        results = compute_metrics(predictions, references)
        print("Evaluation Results:")
        for k, v in results.items():
            print(f"{k}: {v}")
        return results
    else:
        print("No reference explanations provided in test.tsv.")
        return None


In [ ]:
test_inference(
    model_path="/kaggle/working/turbo_model_epoch_5.pt",
    tsv_path="/kaggle/input/more-nlp/MORE-PLUS-DATASET/val_df.tsv",
    d_path="/kaggle/input/more-nlp/MORE-PLUS-DATASET/D_val.pkl",
    o_path="/kaggle/input/more-nlp/MORE-PLUS-DATASET/O_val.pkl"
)


Using device: cuda


<ipython-input-33-0830b1097a87>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))
Asking to truncate to ma

Predictions saved to predictions.txt
Paired output saved to pred_vs_actual.txt
Evaluation Results:
ROUGE: {'rouge1': 0.5489737697711203, 'rouge2': 0.39192706168999825, 'rougeL': 0.5234549977446823, 'rougeLsum': 0.5221202713393198}
METEOR: 0.5503156903750273
BLEU-1: 0.5539789563798931
BLEU-2: 0.46924856943166265
BLEU-3: 0.4121957576168172
BLEU-4: 0.36731301887414214
BERTScore_F1: 0.9238782146998814


{'ROUGE': {'rouge1': 0.5489737697711203,
  'rouge2': 0.39192706168999825,
  'rougeL': 0.5234549977446823,
  'rougeLsum': 0.5221202713393198},
 'METEOR': 0.5503156903750273,
 'BLEU-1': 0.5539789563798931,
 'BLEU-2': 0.46924856943166265,
 'BLEU-3': 0.4121957576168172,
 'BLEU-4': 0.36731301887414214,
 'BERTScore_F1': 0.9238782146998814}

In [ ]:
!zip -r /kaggle/working/working_dir.zip /kaggle/working/*


  adding: kaggle/working/nltk_data/ (stored 0%)
  adding: kaggle/working/nltk_data/corpora/ (stored 0%)
  adding: kaggle/working/nltk_data/corpora/wordnet.zip (stored 0%)
  adding: kaggle/working/nltk_data/corpora/omw-1.4.zip (stored 0%)
  adding: kaggle/working/nltk_data/corpora/wordnet/ (stored 0%)
  adding: kaggle/working/nltk_data/corpora/wordnet/cntlist.rev (deflated 76%)
  adding: kaggle/working/nltk_data/corpora/wordnet/data.noun (deflated 70%)
  adding: kaggle/working/nltk_data/corpora/wordnet/adj.exc (deflated 74%)
  adding: kaggle/working/nltk_data/corpora/wordnet/data.adj (deflated 68%)
  adding: kaggle/working/nltk_data/corpora/wordnet/noun.exc (deflated 65%)
  adding: kaggle/working/nltk_data/corpora/wordnet/index.sense (deflated 73%)
  adding: kaggle/working/nltk_data/corpora/wordnet/index.adj (deflated 71%)
  adding: kaggle/working/nltk_data/corpora/wordnet/LICENSE (deflated 50%)
  adding: kaggle/working/nltk_data/corpora/wordnet/citation.bib (deflated 25%)
  adding: kag